In [1]:
import os 
os.chdir("C:\\Users\\Pieter-Jan\\Documents\\Work\\Candriam\\nlp\\ESG\\topic_sentiment_analysis")

In [5]:
import modules.preprocessing as preprocess

from contextualized_topic_models.models.ctm import CTM
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file, TextHandler
from contextualized_topic_models.datasets.dataset import CTMDataset
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import common_texts
from gensim.models import ldamodel 
import os
import pickle

2020-11-06 15:51:30.004 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary(0 unique tokens: [])
2020-11-06 15:51:30.006 INFO    gensim.corpora.dictionary: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [3]:
doc = "https://www.nvidia.com/content/dam/en-zz/Solutions/documents/FY2019-NVIDIA-CSR-Social-Responsibility.pdf"
doc_txt = preprocess.extract_content(url=doc)

In [34]:
text_proc = preprocess.extract_statements(doc_txt)

In [24]:
import pandas as pd

In [35]:
pd.DataFrame(text_proc, columns=["doc"]).to_csv('output/test.txt', sep='\t', index=False)

In [36]:
from contextualized_topic_models.utils.preprocessing import SimplePreprocessing

In [37]:
documents = [line.strip() for line in open("output/test.txt").readlines()]
sp = SimplePreprocessing(documents)
preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()

In [47]:
pd.DataFrame(preprocessed_documents, columns=["doc"]).to_csv('output/test_processed.txt', sep='\t', index=False)

In [48]:
handler = TextHandler('output/test_processed.txt')
handler.prepare()

In [52]:
training_bert = bert_embeddings_from_file('output/test_processed.txt', "distilbert-base-nli-mean-tokens", batch_size=50)
with open("saved_embeddings", "wb") as filino:
        pickle.dump(training_bert, filino)

2020-11-06 16:21:12.929 INFO    root: Load pretrained SentenceTransformer: distilbert-base-nli-mean-tokens
2020-11-06 16:21:12.931 INFO    root: Did not find folder distilbert-base-nli-mean-tokens. Assume to download model from server.
2020-11-06 16:21:12.933 INFO    root: Load SentenceTransformer from folder: C:\Users\Pieter-Jan/.cache\torch\sentence_transformers\sbert.net_models_distilbert-base-nli-mean-tokens
2020-11-06 16:21:14.784 INFO    root: Use pytorch device: cpu


In [53]:
training_dataset = CTMDataset(handler.bow, training_bert, handler.idx2token)

In [61]:
ctm = CTM(input_size=len(handler.vocab), bert_input_size=768, inference_type="combined", n_components=10, num_epochs=50)
ctm.fit(training_dataset) 

Settings: 
                   N Components: 10
                   Topic Prior Mean: 0.0
                   Topic Prior Variance: 0.9
                   Model Type: prodLDA
                   Hidden Sizes: (100, 100)
                   Activation: softplus
                   Dropout: 0.2
                   Learn Priors: True
                   Learning Rate: 0.002
                   Momentum: 0.99
                   Reduce On Plateau: False
                   Save Dir: None
Epoch: [1/50]	Samples: [197/9850]	Train Loss: 546.7924618793623	Time: 0:00:07.156486
Epoch: [2/50]	Samples: [394/9850]	Train Loss: 543.0474958855489	Time: 0:00:06.780657
Epoch: [3/50]	Samples: [591/9850]	Train Loss: 542.0321949605409	Time: 0:00:06.640990
Epoch: [4/50]	Samples: [788/9850]	Train Loss: 539.687543375238	Time: 0:00:06.570036
Epoch: [5/50]	Samples: [985/9850]	Train Loss: 535.8946923580266	Time: 0:00:06.586986
Epoch: [6/50]	Samples: [1182/9850]	Train Loss: 534.4415524865165	Time: 0:00:06.443135
Epoch: [7/50

In [63]:
ctm.get_topic_lists(10)

[['emissions',
  'scope',
  'ghg',
  'energy',
  'waste',
  'data',
  'metric',
  'gas',
  'environment',
  'total'],
 ['compliance',
  'report',
  'code',
  'standards',
  'rba',
  'presented',
  'signicant',
  'assessment',
  'chain',
  'disclosures'],
 ['use',
  'materials',
  'silicon',
  'using',
  'reducing',
  'include',
  'efcient',
  'certications',
  'valley',
  'new'],
 ['practices',
  'provide',
  'around',
  'industry',
  'improve',
  'define',
  'iso',
  'become',
  'users',
  'teams'],
 ['meetings',
  'oversees',
  'subject',
  'hold',
  'committee',
  'reports',
  'commission',
  'staff',
  'respond',
  'matter'],
 ['requirements',
  'risk',
  'rba',
  'minerals',
  'compliance',
  'priority',
  'validated',
  'suppliers',
  'transparency',
  'critical'],
 ['nvidia',
  'gpu',
  'computing',
  'platform',
  'gpus',
  'supercomputers',
  'worlds',
  'computer',
  'cars',
  'architecture'],
 ['women',
  'minority',
  'recruiting',
  'female',
  'technical',
  'conference',